In [1]:
DATA_NAME = 'syntheticMRI2D-coronal' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_diagonal_wavelet_coronal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,79740.57,2.41,0.09,-1.02,-0.06,-0.05,0.00
4,2.41,129411.32,2.88,0.67,0.77,-0.12,-0.01
5,0.09,2.88,38391.16,0.16,-0.15,0.08,0.01
6,-1.02,0.67,0.16,12669.47,-0.12,-0.02,0.01
7,-0.06,0.77,-0.15,-0.12,1314.64,0.01,0.00
8,-0.05,-0.12,0.08,-0.02,0.01,56.89,-0.00
9,0.00,-0.01,0.01,0.01,0.00,-0.00,2.23


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,0.00002,0.00000,-0.00003,-0.00001,-0.00002,0.00001
4,0.00002,1.00000,0.00004,0.00002,0.00006,-0.00004,-0.00002
5,0.00000,0.00004,1.00000,0.00001,-0.00002,0.00005,0.00002
6,-0.00003,0.00002,0.00001,1.00000,-0.00003,-0.00003,0.00005
7,-0.00001,0.00006,-0.00002,-0.00003,1.00000,0.00002,0.00001
8,-0.00002,-0.00004,0.00005,-0.00003,0.00002,1.00000,-0.00000
9,0.00001,-0.00002,0.00002,0.00005,0.00001,-0.00000,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

5.707288493154093

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.43749169e+09 8.81838718e+08 1.95100620e+08 1.98935769e+07
 1.91853986e+05 2.69992402e+02 5.22693908e-31]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,-0.155995,0.986527,-0.047193,-0.014162,-0.001448,-0.000064,-0.000003
1,0.979594,0.148119,-0.131894,-0.032401,-0.003259,-0.000142,-0.000006
2,0.119696,0.064672,0.985836,-0.097679,-0.008762,-0.000377,-0.000015
3,0.041279,0.025120,0.091504,0.993975,-0.036095,-0.001541,-0.000060
4,0.005502,0.003383,0.011437,0.034879,0.999069,-0.021733,-0.000849
5,0.000357,0.000220,0.000737,0.002243,0.021609,0.998996,-0.039181
6,0.000028,0.000017,0.000058,0.000175,0.001694,0.039154,0.999232


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.1559951367375911,
 0.851880642367418,
 0.014164112796533956,
 0.00602470629590357,
 0.0009314899899485196,
 0.0010044370480131182,
 0.0007682468773073658]